#### 라이브러리 불러오기

In [14]:
import os
import bayes_opt as bo

#### 최적화 대상 함수 생성

In [2]:
def black_box_func(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

#### 최적화

##### 준비물
* 최적화 대상 함수
* 해당 함수의 매개변수 및 매개변수의 min/max값(경계)

In [3]:
SEED = 1

In [4]:
# 매개변수 경계(min/max) 설정
pbounds = {
    'x': (-2, 4), 
    'y': (-3, 3)
}

##### 최적화 객체 생성

In [5]:
optimizer = bo.BayesianOptimization(
    f=black_box_func,       # 최적화 대상 함수
    pbounds=pbounds,        # parameters bounds
    verbose=2,              # 0: 아무것도 출력 안함, 
                            # 1: 최대값이 관찰된 경우만 출력
                            # 2: 모두 출력
    random_state=SEED
)

##### 최적화 실행
* maximize()함수는 최대값을 찾는 최적화 함수이며, 
* 최소값을 찾기 위해서는 대상 함수의 리턴값을 '-'처리해 사용한는 방식으로 처리됨
* (대상함수의 값을 변경할 수 없는 경우에는 어떻게 해야 할지는 나중에 검토)

###### 주요 인자
* n_iter: 몇 단계의 최적화를 수행할 지 결정(단계가 많을 수록 성능이 좋아짐)
* init_points: 무작위 탐사 지점 설정(탐사 공간을 다양화 해줌, 성능 좋아짐)

In [6]:
optimizer.maximize(init_points=2, n_iter=3)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | 0.6442    | 0.5021    | 1.322     |
| 2         | -7.776    | -1.999    | -1.186    |
| 3         | 0.5467    | 0.4431    | 1.507     |
| 4         | -10.83    | 3.392     | 1.571     |
| 5         | -4.254    | -1.697    | 2.541     |


##### 최적화 결과 확인

* 최적화 값 확인

In [7]:
print(optimizer.max)

{'target': 0.6442135807665734, 'params': {'x': 0.5021320282154438, 'y': 1.3219469606529488}}


* 최적화 작업중 검색된 모든 매개변수 목록 조회

In [8]:
for idx, res in enumerate(optimizer.res):
    print(f'Iter {idx}: {res}')

Iter 0: {'target': 0.6442135807665734, 'params': {'x': 0.5021320282154438, 'y': 1.3219469606529488}}
Iter 1: {'target': -7.775871430063692, 'params': {'x': -1.9993137510959307, 'y': -1.1860045642089614}}
Iter 2: {'target': 0.5467184507933587, 'params': {'x': 0.4430853409709734, 'y': 1.506909192877065}}
Iter 3: {'target': -10.832958608004521, 'params': {'x': 3.39212390625311, 'y': 1.571361542834887}}
Iter 4: {'target': -4.253669151873698, 'params': {'x': -1.696700151823901, 'y': 2.5410638360153834}}


#### 최적화 파라메터 값 변경

In [9]:
# 파레메터 값 변경
optimizer.set_bounds(new_bounds={'x': (-2, 3)})
# 최적화 수행 방법 변경
optimizer.maximize(init_points=0, n_iter=5)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 6         | -4.269    | 1.118     | -1.005    |
| 7         | 0.6502    | -0.5735   | 0.8556    |
| 8         | -24.0     | 3.0       | -3.0      |


| 9         | -4.563    | 1.25      | 3.0       |
| 10        | -0.2753   | -0.01698  | -0.1291   |


#### 유추 가능한 파라메터 적용
* 이 부분이 오류가 있어 잠시 주석처리
* 대상 함수의 최댓값이 있는 매개변수 공간(범위)에 대한 예측이 가능한 경우, 옵티마이져가 특정 지점을 조사하도록 지정 가능

In [10]:
# optimizer.probe(
#     params={'x': 0.5, 'y': 0.7},
#     # 바로 실행되지 않고 다음 maximize()가 실행될 때 실행됨
#     lazy=True,          
# )

* 또는 변수없이 바로 사용 가능

In [11]:
# 변수 확인
# optimizer.space.keys

In [12]:
# 위 Key 순서대로 입력
# optimizer.probe(params=[0.5, 0.7], lazy=True)

* 최적화 실행

In [13]:
# optimizer.maximize(int_points=0, n_iter=0)

#### 로그 저장
일반적으로 객체 선언시 `verbose>0`으로 하여 진행사항을 확인할 수 있지만, 로깅 및 알림에 대한 더 많은 제어가 필요한 경우 `옵저버`를 사용할 수 있음(자세한 사항은 고급편에서 다름).<br/>
여기서는, 간단하게 파일에 진행사항을 저장하고 로드하는 `JSONLogger 객체 사용법`만 설명함.

> 로깅을 위한 `옵저버`의 동작원리:
* 옵저버 인스턴스 생성
* 옵티마이저는 특정 상황을 이벤트로 발송하면 옵저버가 수신함

In [15]:
LOG_FILE_PATH = os.path.join(os.getcwd(), 'logs', '01_basic_ture.log.json')
logger = bo.logger.JSONLogger(path=LOG_FILE_PATH)

In [18]:
# 옵티마이저는 새 매개변수-타겟 조합을 얻을 때 마다 
# 'Events.OPTIMIZATION_STEP' 이벤트를 발생
# 일반적인 로그는 이 정도면 됨.

# 즉 로그저장 객체를 만들고, 옵티마이져에 등록하면, 
# 이후 수행되는 optimizer에서 해당 이벤트가 발생하면 로그에 저장됨
optimizer.subscribe(bo.event.Events.OPTIMIZATION_STEP, logger)

In [19]:
# 로그 저장을 위해 optimizer 실행
optimizer.maximize(init_points=2, n_iter=3)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 11        | -12.48    | -1.266    | -2.446    |
| 12        | -3.854    | -1.069    | -0.9266   |
| 13        | 0.9851    | -0.03702  | 1.116     |
| 14        | 0.5063    | -0.4385   | 1.549     |
| 15        | 0.7965    | 0.3043    | 0.6669    |


#### 로그 불러오기
로그를 불러올 때는 새로운 옵티마이저로 불러오며, 옵티마이저를 정의할 때 기존 옵티마이저와 동일하게 작성하지 않아도 됨

In [21]:
new_optimizer = bo.BayesianOptimization(
    f=black_box_func,
    pbounds={'x': (-2, 2), 'y': (-2, 2)},
    verbose=2,
    random_state=SEED
)

In [22]:
bo.util.load_logs(new_optimizer, logs=[LOG_FILE_PATH])

In [23]:
new_optimizer.max

{'target': 0.9850967277901611,
 'params': {'x': -0.037021323864450194, 'y': 1.1163301069764935}}

In [24]:
for idx, res in enumerate(new_optimizer.res):
    print(f'Iteration {idx}: {res}')

Iteration 0: {'target': -12.4780129010131, 'params': {'x': -1.2662205459144347, 'y': -2.445968431387213}}
Iteration 1: {'target': -3.8540423116243687, 'params': {'x': -1.0686989431116456, 'y': -0.9266356377417138}}
Iteration 2: {'target': 0.9850967277901611, 'params': {'x': -0.037021323864450194, 'y': 1.1163301069764935}}
Iteration 3: {'target': 0.5062783386618208, 'params': {'x': -0.4384536935569158, 'y': 1.54907196244616}}
Iteration 4: {'target': 0.7964929710439073, 'params': {'x': 0.30425930287176284, 'y': 0.6669334817606525}}
